In [61]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time
import numpy as np
import argparse, pprint


from matplotlib import pyplot as plt
import matplotlib
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
def pplot(img,num = 0):
    plt.figure(num)
    plt.imshow(img)
    plt.subplots_adjust(wspace=0.0,hspace=0.0,left=0.0,right=1.0,top=1.0, bottom=0.0)
    plt.show()
    
def test_intersection(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    iou = interArea / float(boxAArea + boxBArea - interArea)
    return iou

def union(a,b):
    x = min(a[0], b[0])
    y = min(a[1], b[1])
    w = max(a[0]+a[2], b[0]+b[2]) - x
    h = max(a[1]+a[3], b[1]+b[3]) - y
    return (x, y, w, h)

def spread_image(image, percentile=1.1):
    copy = image.copy()
    summed = np.sum(copy, axis=1)
    total = np.sum(summed)
    percentile_limit = total / percentile
    running_total = 0
    for i in range(summed.shape[0]):
        running_total += summed[i]
        if running_total > percentile_limit:
            break

    reshape_factor = copy.shape[0] / float(i)
    print(copy.shape)
    copy = cv2.resize(copy, (copy.shape[1], int(copy.shape[0]*reshape_factor)), interpolation=cv2.INTER_NEAREST)
    print(copy.shape)
#     tmpI = int(copy.shape[0]/reshape_factor)+1
#     print(tmpI)
#     copy = copy[:tmpI,:]
#     print(copy.shape)
    return copy, reshape_factor

def despread_image(image,reshape_factor):
    copy = image.copy()
    print(copy.shape)
    sz1 = copy.shape[1]
    sz2 = int(copy.shape[0]/reshape_factor)+2
#     sz2 = int(int(copy.shape[0]/reshape_factor)*reshape_factor)
    
    copy = cv2.resize(copy, (sz1, sz2), interpolation=cv2.INTER_NEAREST)
    sz3 = int(copy.shape[0]*reshape_factor)
#     copy = copy[:sz3,:]
    print(sz1,sz2,sz3)
    return copy

In [185]:
from src.UVMappingPipeline import UVMappingPipeline
from src.VBOATS import VBOATS
from utils.sys_utils import *
from utils.seg_utils import *
from utils.uv_mapping_utils import *

pipe = UVMappingPipeline()
vboat = VBOATS()

In [227]:
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/test_cases")
# found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/test/frames")
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/obstacles")
# found_imgs, _ = grab_dir_images("/home/hunter/data/rosbags/extracted/depth")

# found_imgs, _ = grab_dir_images("/home/hyoung/Documents/rosbags/extracted/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/devel/vision_playground/test/iai/obstacles")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/tmp/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/distance_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/angle_test/depth")
found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/misc_test/depth")
sFImgs = sorted(found_imgs)

In [228]:
style = {'description_width': 'initial'}
layStretch = Layout(flex='1 1 auto')
layCenter = Layout(align_self='center',display = 'flex',align_items='stretch',flex='1 1 auto')
layWidth = Layout(width="auto",align_self='center')

imgPathSelector = widgets.Select(options=sFImgs,value=str(sFImgs[0]),description='File:', layout=Layout(width='100%'))

lblSz1 = widgets.Label(value="Deadzone Sizes (U,V):", layout=layCenter)
sz1Selector = widgets.IntSlider(value=0,min=0,max=20,step=1,description=' ', layout = layCenter)
sz2Selector = widgets.IntSlider(value=3,min=0,max=20,step=1,description=' ', layout = layCenter)

lblSz2 = widgets.Label(value="Strip[0] Deadzone Sizes (U,V):", layout=layCenter)
sz3Selector = widgets.IntSlider(value=4,min=0,max=20,step=1,description=' ', layout = layStretch)
sz4Selector = widgets.IntSlider(value=5,min=0,max=20,step=1,description=' ', layout = layStretch)

lblU = widgets.Label(value="UMap Strip Thresholds (0,1):", layout=layCenter)
u1threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout= layStretch)
# u1threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ',layout= layStretch)
u2threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout = layStretch)

lblV = widgets.Label(value="VMap Strip Thresholds (0,1):", layout=layCenter)
v1threshSelector = widgets.IntSlider(value=9,min=0,max=255,step=1,description=' ',layout = layStretch)
v2threshSelector = widgets.IntSlider(value=70,min=0,max=255,step=1,description=' ',layout = layStretch)

lblD = widgets.Label(value="Deadzone Thresholds (U1, U2, V):", layout=layCenter)
d1threshSelector = widgets.FloatSlider(value=0.4,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d2threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d3threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)

lblK = widgets.Label(value="Kernel Sizes:", layout=layCenter)
k11Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description='k11', layout = layStretch)
k12Selector = widgets.IntSlider(value=2,min=0,max=100,step=1,description='k12', layout = layStretch)
k21Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k21', layout = layStretch)
k22Selector = widgets.IntSlider(value=5,min=0,max=100,step=1,description='k22', layout = layStretch)
k31Selector = widgets.IntSlider(value=75,min=0,max=100,step=1,description='k31', layout = layStretch)
k32Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k32', layout = layStretch)

win1Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description='win1', layout = layStretch)
win1ThreshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)


flag1 = widgets.Checkbox(value=True, description='Use Test Pipeline', layout = layCenter)
flag2 = widgets.Checkbox(value=False, description='Show Computational Timing', layout = layCenter)
flag3 = widgets.Checkbox(value=True, description='Use Morphs', layout = layCenter)

In [229]:
from utils.uv_mapping_utils import *
def thresholding(_img, dx=0,dy=3, sz1=5,sz2=10,tu0=0.15,tu1=0.1,tv0=5, tv1=70, tdu = 0.4, tdu2 = 0.1, tdv = 0.15,
    k11=10,k12=2,k21=50,k22=5,k31=75,k32=1, win1=5, win1thr = 0.1, searchWin=10, flag_test=True, flag_timing=False,flag_morph=True
):
    if(flag_test):
        vboat.dW = sz2;        vboat.dH = sz1
        vboat.dead_x = dx;     vboat.dead_y = dy
        vboat.dThs = [tdu,tdu2,win1thr,tdv]
        vboat.dHplus = win1
        
        vboat.kszs[0] = (k11,k12)
        vboat.kszs[1] = (k21,k22)
        vboat.kszs[2] = (k31,k32)
        vboat.flag_morphs = flag_morph
        
        vboat.vmap_search_window_height = searchWin
        
        vboat.pipelineTest(_img, threshU1=tu0,threshU2=tu1, threshV1=tv0, threshV2=tv1, timing=flag_timing)
        display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)
        display_obstacles = cv2.applyColorMap(display_obstacles,cv2.COLORMAP_JET)
        
        for ob in vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(255,255,255),1)
        
        overlay = make_uv_overlay(display_obstacles,vboat.umap_deadzoned,vboat.vmap_raw)
        overlay = cv2.cvtColor(overlay,cv2.COLOR_BGR2RGB)
        pplot(overlay,'Overlay')

        dispU1,dispU2,compU = vboat.umap_displays()
        dispV1,dispV2,compV = vboat.vmap_displays()

        dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
        dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
        pplot(dispU2,'Umap Display')
        pplot(dispV2,'Vmap Display')
        
        distances = []
        angles = []
        umap = vboat.umap_raw
        xs = vboat.xBounds
        ds = np.array(vboat.dbounds)
        obs = vboat.obstacles
        nObs = len(ds)
        if(nObs is not 0):
            for i in range(nObs):
                disparities = ds[i]
                us = [obs[i][0][0], obs[i][1][0]]
                vs = [obs[i][0][1], obs[i][1][1]]
                z,ux,uy,uz = vboat.calculate_distance(umap,us,disparities,vs)

                theta = math.acos((uz/z))

                distances.append(z)
                angles.append(theta)
                print("Obstacle [%d] Distance, Angle: %.3f, %.3f" % (i,z,np.rad2deg(theta)))
        
    else:
        vboat.pipeline(_img, threshU1=tu0,threshU2=tu1, threshV1=tv0, threshV2=tv1, timing=flag_timing)
        display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)

        for ob in vboat.obstacles:
            cv2.rectangle(display_obstacles,ob[0],ob[1],(150,0,0),1)
        pplot(display_obstacles,'Pipeline Obstacles')

In [230]:
%reload_ext autoreload
plt.close('all')

p = interactive(thresholding,
    _img = imgPathSelector, dx = sz1Selector, dy=sz2Selector, sz1=sz3Selector,sz2=sz4Selector,
    tu0 = u1threshSelector,tu1 = u2threshSelector, tv0 = v1threshSelector, tv1 = v2threshSelector,
    tdu = d1threshSelector, tdu2=d2threshSelector, tdv = d3threshSelector, k11=k11Selector,k12=k12Selector,
    k21=k21Selector,k22=k22Selector,k31=k31Selector,k32=k32Selector,win1=win1Selector,win1thr=win1ThreshSelector,
    searchWin=(0,40,1), flag_test = flag1, flag_timing = flag2, flag_morph = flag3
)

rows = []
rows.append(HBox([p.children[0]]))
rows.append(HBox([lblSz1,p.children[1], p.children[2]], layout = layStretch))
rows.append(HBox([lblSz2,p.children[3], p.children[4]], layout = layStretch))
rows.append(HBox([lblU, p.children[5], p.children[6]], layout = layStretch))
rows.append(HBox([lblV, p.children[7], p.children[8]], layout = layStretch))
rows.append(HBox([lblD, p.children[9], p.children[10], p.children[11]], layout = layStretch))
rows.append(HBox([p.children[12], p.children[13]], layout = layStretch))
rows.append(HBox([p.children[14], p.children[15]], layout = layStretch))
rows.append(HBox([p.children[16], p.children[17]], layout = layStretch))
rows.append(HBox([p.children[18], p.children[19]], layout = layStretch))

rows.append(HBox([p.children[-4], p.children[-3], p.children[-2]], layout = layCenter))

controls = VBox(rows, layout = Layout(display='flex'))
output = p.children[-1]

tabs = widgets.Tab([controls, output])
tabs.set_title(0, 'Parameters')
tabs.set_title(1, 'StdOuts')
display(tabs)

VGFiKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShTZWxlY3QoZGVzY3JpcHRpb249dSdGaWxlOicsIGxheW91dD1MYXlvdXQod2lkdGg9dScxMDAlJyksIG9wdGlvbnPigKY=


In [222]:
import math
np.sqrt(2*2 + 0.965*0.965)

2.2206361701098176

In [197]:
tImg = np.copy(vboat.img)
umap = vboat.umap_raw
vmap = vboat.vmap_raw
deadu = vboat.umap_deadzoned
pv = vboat.vmap_processed
pu = vboat.umap_processed

# pplot(tImg,'Input')
pplot(umap,'Raw Umap')
# pplot(vmap,'Raw Vmap')
pplot(deadu,'Deadzoned Umap')
pplot(pv,'Processed vmap')



# mn = np.mean(tImg)
# st = np.std(tImg[tImg > 10])
# norm = (tImg - mn)/st

# norm = np.expand_dims(norm,axis=-1)

# norm = cv2.normalize(norm,None,0.0,255.0,cv2.NORM_MINMAX)
# norm = np.asarray(norm, dtype=np.uint8)
# nUmap, nVmap,_ = vboat.get_uv_map(norm)

# sub = nUmap - umap 
# pplot(sub,'Sub')

# copy = vmap.copy()
# # copy = cv2.cvtColor(copy,cv2.COLOR_GRAY2BGR)
# copy = np.float32(copy)
# criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
# K = 150
# ret,label,center=cv2.kmeans(copy,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# # Now convert back into uint8, and make original image
# center = np.uint8(center)
# res = center[label.flatten()]
# res2 = res.reshape((vmap.shape))
# pplot(res2,'kmeans')

# nUmap = nUmap**2
# nUmap[nUmap < np.max(nUmap*0.3)] = 0

# pplot(nUmap,'Norm Umap')

# pplot(vboat.umap_processed,'Processed Umap')

# _, tmpU = cv2.threshold(vmap, 10, 255,cv2.THRESH_TOZERO)
# tmpU = cv2.equalizeHist(tmpU)

# pplot(tmpU,1)

In [176]:
dispU1,dispU2,compU = vboat.umap_displays()
dispV1,dispV2,compV = vboat.vmap_displays()

# plt.figure(6)
# plt.imshow(vboat.img)
# plt.show()

dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
plt.figure(0)
plt.imshow(dispU2)
plt.show()

# plt.figure(1)
# plt.imshow(dispU2)
# plt.show()

plt.figure(2)
plt.imshow(vboat.vmap_processed**2)
plt.show()

dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
plt.figure(3)
plt.imshow(dispV2)
plt.show()

# plt.figure(4)
# plt.imshow(dispV2)
# plt.show()

# plt.figure(5)
# plt.imshow(compV)
# plt.show()

In [ ]:
tImg = np.copy(vboat.img)

raw_umap, raw_vmap, dt = vboat.get_uv_map(tImg)

# =========================================================================
dead_y = vboat.dead_y; dead_x = vboat.dead_x

deadzoneU = raw_umap[1:dead_y+1, :]
_, deadzoneU = cv2.threshold(deadzoneU, 80, 255,cv2.THRESH_BINARY)

cv2.rectangle(raw_umap,(0,0),(raw_umap.shape[1],dead_y),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_umap,(0,raw_umap.shape[0]-dead_y),(raw_umap.shape[1],raw_umap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(0,0),(dead_x, raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
cv2.rectangle(raw_vmap,(raw_vmap.shape[1]-dead_x,0),(raw_vmap.shape[1],raw_vmap.shape[0]),(0,0,0), cv2.FILLED)
# =========================================================================

threshs = [11, 10, 30, 40,40,40]
stripsPu = []
stripsU = strip_image(raw_umap, nstrips=len(threshs))

strU1 = stripsU[0]
strU2 = stripsU[1]

for i, strip in enumerate(stripsU):
#     _, tmpStrip = cv2.threshold(strip, threshs[i], 255,cv2.THRESH_BINARY)
    _, tmpStrip = cv2.threshold(strip, threshs[i], 255,cv2.THRESH_TOZERO)
    stripsPu.append(tmpStrip)

kernel = cv2.getStructuringElement(cv2.MORPH_RECT,(15,2))
stripsPu[0] = cv2.morphologyEx(stripsPu[0], cv2.MORPH_CLOSE, kernel)

tK = np.ones((3,2),np.uint8)
copy = stripsPu[1].copy()

stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_CLOSE, kernel)

# stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_OPEN, (2,2))
# stripsPu[1] = cv2.blur(stripsPu[1],(2,2))
tmpp = cv2.morphologyEx(stripsPu[1], cv2.MORPH_GRADIENT, kernel)
# stripsPu[1] = cv2.equalizeHist(stripsPu[1])
# stripsPu[2] = cv2.morphologyEx(stripsPu[2], cv2.MORPH_OPEN, kernel)
pplot(tmpp, 'tmpp')
stripsPu[1] = cv2.morphologyEx(stripsPu[1], cv2.MORPH_BLACKHAT, kernel)

# ============================ CRAP ============================
kernel = np.ones((4,4),np.uint8)

_, tmpStrip = cv2.threshold(stripsPu[1], 1, 255,cv2.THRESH_BINARY)
# _, tmpStrip2 = cv2.threshold(stripsPu[1], 1, 255,cv2.THRESH_BINARY_INV)
# dist_transform = cv2.distanceTransform(stripsPu[1],cv2.DIST_L2,5)
dist_transform = cv2.distanceTransform(tmpStrip,cv2.DIST_L2,5)
# dist_transform2 = cv2.distanceTransform(tmpStrip2,cv2.DIST_L2,5)
# pplot(dist_transform, 'dist_transform')
# pplot(dist_transform2, 'dist_transform2')

# erode = cv2.dilate(dist_transform2,kernel,iterations=5)
erode = cv2.erode(dist_transform,kernel,iterations=1)
# stripsPu[1] = np.uint8(erode)

# dist_transform = cv2.distanceTransform(stripsPu[1],cv2.DIST_L2,5)
# erode = cv2.erode(dist_transform,kernel,iterations=2)
# _, sure_fg = cv2.threshold(erode,0.1*erode.max(),255,0)

# sure_bg = cv2.dilate(sure_fg,kernel,iterations=1)
# unknown = cv2.subtract(sure_bg,sure_fg)

# # pplot(dist_transform, 'dist_transform')
# # pplot(sure_fg, 'sure_fg')
# # pplot(erode, 'erode')

# sure_fg = np.uint8(sure_fg)
# _, markers = cv2.connectedComponents(sure_fg)


# markers = markers+1
# # Now, mark the region of unknown with zero
# markers[unknown==255] = 0
# pplot(markers, 'markers')

# tmp = cv2.cvtColor(stripsPu[1],cv2.COLOR_GRAY2BGR)
# markers = cv2.watershed(tmp,markers)
# tmp[markers == -1] = [255,0,0]

# pplot(tmp, 'tmp')
# ============================ CRAP ============================


hUs,w = stripsPu[0].shape[:2]
blankStrip = np.zeros((hUs-dead_y,w),dtype=np.uint8)
deadzone_mask = np.concatenate((deadzoneU,blankStrip), axis=0)
# try: deadzone_mask = cv2.cvtColor(deadzone_mask, cv2.COLOR_GRAY2BGR)
# except: print("[WARNING] ------------  Unnecessary Deadzone Image Color Converting")

kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,(50,5))
deadzone_mask = cv2.morphologyEx(deadzone_mask, cv2.MORPH_CLOSE, kernelD)

# print(stripsPu[0].shape, deadzone_mask.shape)
stripsPu[0] = cv2.addWeighted(stripsPu[0], 1.0, deadzone_mask, 1.0, 0)
pplot(stripsPu[0], 'stripsPu[0] After')

tmpMax = np.max(stripsPu[0])
# print(tmpMax)
dead_strip = stripsPu[0][0:14, :]
rest_strip = stripsPu[0][14:stripsPu[0].shape[0], :]

dead_strip[dead_strip < tmpMax*0.4] = 0
rest_strip[rest_strip < tmpMax*0.1] = 0
stripsPu[0] = np.concatenate((dead_strip,rest_strip), axis=0)


# pplot(stripsPu[1], 'stripsPu[1] After')



pplot(deadzone_mask, 'Deadzone Mask')
   
pUmap = np.concatenate(stripsPu, axis=0)

cnt_threshs = [55.0,50.0,80.0,40.0,40.0,40.0]
contours = []
for i, strip in enumerate(stripsPu):
    contours += vboat.find_contours(strip, cnt_threshs[i], offset=(0,hUs*i))
# contours = fCnts1 + fCnts2 + fCnts3 + fCnts4 + fCnts5 + fCnts6
print(len(contours))

disp = pUmap.copy()
for cnt in contours:
    cv2.drawContours(disp, [cnt], 0, (255,0,255), 1)
    
pplot(disp, 'Filtered Umap')

xLims, dLims, _ = vboat.extract_contour_bounds(contours)
# ==============================================================================
#                           V MAP
# ==============================================================================
threshsV = [5, 20, 40,40,40]
stripsV = strip_image(raw_vmap, nstrips=len(threshsV), horizontal_strips=False)

pplot(stripsV[0], 'stripsPv[0] Before')
# pplot(stripsV[1], 'stripsPv[1] Before')

stripsPv = []
for i, strip in enumerate(stripsV):
#     _, tmpStrip = cv2.threshold(strip, threshsV[i], 255,cv2.THRESH_BINARY)
    _, tmpStrip = cv2.threshold(strip, threshsV[i], 255,cv2.THRESH_TOZERO)
    stripsPv.append(tmpStrip)
    
tmpMax = np.max(stripsPv[0])
# print(tmpMax)
stripsPv[0][stripsPv[0] < np.max(stripsPv[0]*0.035)] = 0

top_strip = stripsPv[0][0:100, :]
bot_strip = stripsPv[0][100:stripsPv[0].shape[0], :]
    
top_strip[top_strip < tmpMax*0.1] = 0
bot_strip[bot_strip < tmpMax*0.05] = 0
stripsPv[0] = np.concatenate((top_strip,bot_strip), axis=0)

# tmpMax = np.max(stripsPv[1])
tH = stripsPv[1].shape[0]
top_strip = stripsPv[1][0:tH/2, :]
bot_strip = stripsPv[1][tH/2:tH, :]

top_strip[top_strip < tmpMax] = 0
# bot_strip[bot_strip < tmpMax*0.05] = 0
stripsPv[1] = np.concatenate((top_strip,bot_strip), axis=0)    
    
    
# pplot(stripsPv[0]**2, 'stripsPv[0] After')
# pplot(stripsPv[1]**2, 'stripsPv[1] After')
    
newV = np.concatenate(stripsPv, axis=1)
tmpV = cv2.cvtColor(newV, cv2.COLOR_GRAY2BGR)
mask, maskInv,mPxls, ground_wins,_ = vboat.get_vmap_mask(tmpV, maxStep = 15)
vmap = cv2.bitwise_and(newV,newV,mask=maskInv)
pplot(vmap*10, 'Vmap After')
# =========================================================================

obs, obsU, ybounds, dbounds, windows, nObs = vboat.find_obstacles(vmap, dLims, xLims)

dispObs = cv2.cvtColor(tImg, cv2.COLOR_GRAY2BGR)

for ob in obs:
    cv2.rectangle(dispObs,ob[0],ob[1],(150,0,0),1)
    
print(len(obs))
pplot(dispObs, 'Obstacles')
# pplot(strU1, 'stripsPu[0] Before')
# pplot(strU2, 'stripsPu[1] Before')

In [ ]:
sprImg,fact = spread_image(strU1)
sub1 = strU1 - sprImg[:42,:]
pplot(sub1,'Spread U1')

desprdImg = despread_image(sprImg,fact)
print(desprdImg.shape)
sub2 = strU1 - desprdImg[:42,:]
pplot(sub2,'Spread U2')
pplot(desprdImg,'DeSpread U2')


In [ ]:
ellipses = []
disp = np.copy(vboat.umap_processed)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for cnt in vboat.filtered_contours:
    ellipse = cv2.fitEllipse(cnt)
    ellipses.append(ellipse)
    cv2.ellipse(disp,ellipse,(0,255,255),1)
    
plt.figure(1)
plt.imshow(disp)
plt.show()

In [ ]:
img = np.copy(vboat.img)
print(vboat.obstacles[0])
win = vboat.obstacles[0]
print(win[0][0], win[1][0])

rectsTypeA = [] # x0, y0, x1, y1
rectsTypeB = [] # x, y, w, h
for win in vboat.obstacles:
    
    wid = win[1][0] - win[0][0]
    h = win[1][1] - win[0][1]
    cx = win[0][0] + wid/2
    cy = win[0][1] + h/2
    
    tmpA = [win[0][0], win[0][1], win[1][0], win[1][1]]
    tmpB = [cx, cy, wid, h]
    rectsTypeA.append(tmpA)
    rectsTypeB.append(tmpB)


# cv2.circle(img,(pxl[0],pxl[1]),2,(255,0,255),-1)

print(wid, h)
print(cx, cy)
print(rectsTypeB)
print(vboat.nObs)

In [ ]:
n = vboat.nObs
idcs = []
pick = []
boxes = np.array(rectsTypeA)
x1 = boxes[:,0]
y1 = boxes[:,1]
x2 = boxes[:,2]
y2 = boxes[:,3]
area = (x2 - x1 + 1) * (y2 - y1 + 1)
idxs = np.argsort(y2)
overlapThresh = 0.05
supressed = []
top = []
non = []
print(idxs)
while len(idxs) > 0:
    # grab the last index in the indexes list, add the index
    # value to the list of picked indexes, then initialize
    # the suppression list (i.e. indexes that will be deleted)
    # using the last index
    last = len(idxs) - 1
    i = idxs[last]
    pick.append(i)
    suppress = [last]
    # loop over all indexes in the indexes list
    print(i)
    overlaps = False
    for pos in xrange(0, last):
        # grab the current index
        j = idxs[pos]

        # find the largest (x, y) coordinates for the start of
        # the bounding box and the smallest (x, y) coordinates
        # for the end of the bounding box
        xx1 = max(x1[i], x1[j])
        yy1 = max(y1[i], y1[j])
        xx2 = min(x2[i], x2[j])
        yy2 = min(y2[i], y2[j])

        # compute the width and height of the bounding box
        w = max(0, xx2 - xx1 + 1)
        h = max(0, yy2 - yy1 + 1)

        # compute the ratio of overlap between the computed
        # bounding box and the bounding box in the area list
        
        overlap = float(w * h) / area[j]

        print("Window %d Overlaps %.4f with Rectangle %d" % (j,overlap, i) )
        # if there is sufficient overlap, suppress the
        # current bounding box
        if overlap > overlapThresh:
#             suppress.append(pos)
            supressed.append(pos)
            top.append([pos,i])
            overlaps = True

    # delete all indexes from the index list that are in the
    # suppression list
    print("suppressing idxs", suppress)
#     supressed.append(suppress)
    idxs = np.delete(idxs, suppress)
    if(not overlaps):
        non.append(last)

rectangles = [] 
for a,b in top:
    rect = union(rectsTypeA[a],rectsTypeA[b])
    rectangles.append(rect)
    

print(top)
print(non)
print(supressed)
mask = np.isin(non, supressed, invert=True)
remain = np.where(mask)
print(remain)
rectangles.append(rectsTypeA[remain[0][0]])

print(rectangles)
for i in range(n):
    has_overlap = False
    for j in range(n):
        if(i is not j):
            iou = test_intersection(rectsTypeA[i],rectsTypeA[j])
            if(iou >= 0.05):
                has_overlap = True
                idcs.append([i, j])
                print("IoU for Rect %d and Rect %d = %.3f" % (i, j, iou))
    if(not has_overlap):
        print("Rect %d has no overlapping segments" % i)

In [ ]:
disp = np.copy(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
for x1,y1,x2,y2 in boxes[pick]:
    cv2.rectangle(disp,(x1,y1),(x2,y2),(0,255,255), 1)
    
plt.figure(10)
plt.imshow(disp)
plt.show()

In [ ]:
disp = np.zeros_like(vboat.img)
disp = cv2.cvtColor(disp,cv2.COLOR_GRAY2BGR)
masks = []
for win in vboat.obstacles:
    mask = np.copy(disp)
    cv2.rectangle(mask,win[0],win[1],(255,255,255), cv2.FILLED)
    mask = cv2.cvtColor(mask,cv2.COLOR_BGR2GRAY)
    masks.append(mask)
    

tmp = cv2.bitwise_and(vboat.img,vboat.img, mask=masks[0])

tmpMean = np.mean(tmp)
print(tmpMean/0.001)
plt.figure(2)
plt.imshow(tmp)
plt.show()